# imports

In [1]:
import pandas as pd
import numpy as np
import re
import tqdm
import itertools
import json
import inflect

In [2]:
def print_bold(str):
    print('\033[1m' + str + '\033[0m')

# files paths

In [3]:
#where to find useful data
food_des_path = "./../data/usda/FOOD_DES.txt"
food_groups_path = "./../data/usda/FD_GROUP.txt"
nut_data_path = "./../data/usda/NUT_DATA.txt"
nut_def_path = "./../data/usda/NUTR_DEF.txt"

all_paths = [food_des_path, food_groups_path, nut_data_path, nut_def_path]

# Load and filter food group description

### info about the different food groups contained in the USDA database

In [4]:
columns = ["food_group_id", "food_group_name"]

food_groups = pd.read_csv(food_groups_path, sep="^", encoding="ISO-8859-1", names=columns, header=None)

food_groups.set_index("food_group_id", inplace=True)

food_groups

,food_group_name
food_group_id,
100,Dairy and Egg Products
200,Spices and Herbs
300,Baby Foods
400,Fats and Oils
500,Poultry Products
600,"Soups, Sauces, and Gravies"
700,Sausages and Luncheon Meats
800,Breakfast Cereals
900,Fruits and Fruit Juices


### we decide to drop :

>* Baby food (300)
>* dressing in 400
>* soup in 600
>* Breakfast Cereals (800)
>* Beverages (1400) except water
>* Baked products (1800) 
>* Sweets (1900) except Baking products (chocolate)
>* Fast Foods (2100)
>* Meals, Entrees, and Side Dishes (2200)
>* Snacks (2500)
>* Restaurant food (3600)


# Load and clean food description table

#### Info about each individual product

In [5]:
#take a description-like syntax and return the list of words, filtering  no negation
def split_des_in_list(des) :
    
    cats = re.sub("[()]", "", des).strip().lower().split(',')    
    final_list = []
    
    for c in cats :        
        words_list = c.strip().split(" ")
        
        if (("no" not in words_list) and ("without" not in words_list)) :
            final_list.extend([c.strip().lower() for c in words_list if c != "with"])
    
    return final_list
    
    
#return [w.strip() for c in des.split(",") for w in c.strip().lower().split(' ')]

#singularize a word if plural
def singularize_word(x) :
    if engine.singular_noun(x) :
        return engine.singular_noun(x)
    else :
        return x
        
#clean the description(lowercases, strips, singularization)
def format_long_des(x) :
        
    if str(x) == "nan" :
        return ""    
    
    split = split_des_in_list(x)
    
    #decompose description
    words = [c for c in split]
    
    #singularize words
    sing_words = [singularize_word(x) for x in words]
    
    #rebuild description
    return str(" ".join(sing_words))

def concat_common_and_des(common, des) :
    if common != "" :
        return common.split(" ") + des.split(" ")
    else :
        return des.split(" ")

In [6]:
#generate singularization engine
engine = inflect.engine()

#columns we want to retrieve from the database files
columns = ["food_id", "food_group_id", "long_description", "common_names"]
use_cols = [0, 1, 2, 4]

#get the info from file
food_des = pd.read_csv(food_des_path, sep="^", encoding="ISO-8859-1", names=columns, usecols=use_cols, header=None)

#display original table format
print_bold("original table format : ")
print(food_des.head())


# generate search_words
food_des['search_words'] = food_des.apply(lambda row : concat_common_and_des(format_long_des(row['common_names']),
                                                                             format_long_des(row['long_description'])),
                                                                             axis=1)

#drop common_names column
food_des = food_des.drop("common_names", axis=1)
            
print_bold("\n\nnew table format")
food_des.head()

original table format : 
   food_id  food_group_id            long_description common_names
0     1001            100              Butter, salted          NaN
1     1002            100  Butter, whipped, with salt          NaN
2     1003            100       Butter oil, anhydrous          NaN
3     1004            100                Cheese, blue          NaN
4     1005            100               Cheese, brick          NaN


new table format


,food_id,food_group_id,long_description,search_words
0,1001,100,"Butter, salted","[butter, salted]"
1,1002,100,"Butter, whipped, with salt","[butter, whipped, salt]"
2,1003,100,"Butter oil, anhydrous","[butter, oil, anhydrou]"
3,1004,100,"Cheese, blue","[cheese, blue]"
4,1005,100,"Cheese, brick","[cheese, brick]"


In [7]:
food_des[food_des["food_id"] == 2014]

,food_id,food_group_id,long_description,search_words
276,2014,200,"Spices, cumin seed","[spice, cumin, seed]"


#### define a search function for mapping recipes ingredients

In [7]:
def search_ingredient(ingredient, print_search_candidates=False) :
    
    #do not penalize the presence of those words, 'table' is for the salt
    non_complexificators = set(["fresh", "raw", "skin", "peel", "whole"])
    
    def search_score(categories, ing_words) :
        
        # singularize search words
        ing_words = set([singularize_word(x) for x in ing_words])
        
        #prioritize matching query terms
        nb_matching = len(ing_words.intersection(set(categories)))
        
        #non_complexificators should not be penalized,ignore them AFTER computing number of matching words
        categories = [c for c in categories if (c not in non_complexificators)]
        
        
        #matching keywords one by one 
        matching = [len(set([x]).intersection(ing_words)) != 0 for x in categories]
            
        
        #first keywords are more important
        weights = np.linspace(2, 1, num=len(matching))
        weights = weights / sum(weights)
        
        #the query should have as many ingredients words as possible
        score = (10 * nb_matching) + sum([c[0] * c[1] for c in zip(matching, weights)])
        
        return score
    
    
    ing_words = set(ingredient.split(" "))       
    
    #compute search score for each entry and sort them by score (descending order)
    food_des["search_score"] = food_des["search_words"].apply(lambda x : search_score(x, ing_words))  
    food_des_sorted = food_des.sort_values(by=['search_score'], ascending=False)

    #print best candidates
    if print_search_candidates :
        print(food_des_sorted.head())

    #best score
    result = food_des_sorted[["food_id", "search_words", "search_score"]].head(1)        

    #check if we found a positive score
    if result["search_score"].values[0] != 0 :
        return result, result["search_score"].values[0]
    else :
        return None, 0
    
    
" ".join(search_ingredient("asparagus", print_search_candidates=True)[0]['search_words'].values[0])

      food_id  food_group_id                               long_description  \
2439    11011           1100                                 Asparagus, raw   
2443    11018           1100                  Asparagus, frozen, unprepared   
2442    11015           1100              Asparagus, canned, drained solids   
2440    11012           1100             Asparagus, cooked, boiled, drained   
2936    11705           1100  Asparagus, cooked, boiled, drained, with salt   

                                   search_words  search_score  
2439                            [asparagu, raw]     11.000000  
2443             [asparagu, frozen, unprepared]     10.444444  
2442         [asparagu, canned, drained, solid]     10.333333  
2440        [asparagu, cooked, boiled, drained]     10.333333  
2936  [asparagu, cooked, boiled, drained, salt]     10.266667  


'asparagu raw'

In [8]:
all_ingredients_ids = {}
total_ing_count = 0
mapped_at_least_one_ing_count = 0
mapped_all_ing_count = 0
ingredients = json.load(open("../generated/ingredients_count.json"))['count']

for k, v in tqdm.tqdm(ingredients.items()) :
    #print(k, v)
    cats, score = search_ingredient(k, print_search_candidates=False)
    total_ing_count += v
     
    if score >= 10 * len(k.split(" ")) :
        mapped_all_ing_count += v
        all_ingredients_ids[k] = int(cats['food_id'].values[0])
                         
        
#save mapping
json.dump(all_ingredients_ids, open("../generated/ing_id_mapping.json", 'w'))

100%|██████████████████████████████████████████████████████████████████████████| 11233/11233 [2:29:24<00:00,  1.13it/s]


> Good, the mapping is done ! How many ingredients were we able to map effectively ?

In [10]:
print_bold("{0:.2f}".format(100 * len(all_ingredients_ids) / len(ingredients)) + "% of ingredients were mapped succesfully (all words matched)")

23.58% of ingredients were mapped succesfully (all words matched)


> It occurs that some ingredients are much more common that others, thus, it is more important to map those ingredients than to map the less frequent ones. What if we weight the success rate given the number of times the ingredients appear in recipes ?

In [13]:
len(all_ingredients_ids)

2649

In [11]:
print_bold("You have {0:.2f}".format(100 * mapped_all_ing_count / total_ing_count) + "% of chance to find a matching database entry for any recipe ingredients entry")

high score : you have 81.07% of chance to find a matching database entry for any recipe ingredients entry


#### save the long_description for food items description purposes

In [19]:
id_describe = dict()
high_mapping = all_ingredients_ids
all_ids = list(set([high_mapping[c] for c in high_mapping]))
for index in tqdm.tqdm(all_ids) :
    id_describe[index] = food_des[food_des['food_id'] == index]['long_description'].values[0]
    
print(len(id_describe))
json.dump(id_describe, open("../generated/id_description.json", 'w'))

100%|████████████████████████████████████████████████████████████████████████████| 1863/1863 [00:00<00:00, 2253.23it/s]


1863


# nutrient and RDI linking

#### import RDI values

In [21]:
rdi = pd.read_excel("../data/RDI.xlsx")
rdi_nutrients = rdi['nutrient'].values
rdi_nutrients

array(['alpha-linoleic acid', 'Biotin', 'Calcium', 'Carbohydrate',
       'Chloride', 'Choline', 'Chromium', 'Copper', 'Fat', 'Fiber',
       'Fluoride', 'Folate', 'Iodine', 'Iron', 'Linoleic acid',
       'Magnesium', 'Manganese', 'Molybdenum', 'Monousaturated fat',
       'Niacin', 'Pantothenic acid', 'Phosphorus', 'Polyunsaturated fat',
       'Potassium', 'Protein', 'Riboflavin', 'Saturated fat', 'Selenium',
       'Sodium', 'Sugar', 'Thiamin', 'Vitamin A', 'Vitamin B6',
       'Vitamin B12', 'Vitamin C', 'Vitamin D', 'Vitamin E', 'Vitamin K',
       'Water', 'Zinc'], dtype=object)

#### Load and filter Nutrient definition

In [22]:
use_cols = [0, 1, 2, 3]

columns = ["nutrient_id", "units", "tagname", "description"]

nut_def = pd.read_csv(nut_def_path, sep="^", encoding="ISO-8859-1", names=columns, usecols=use_cols, header=None)

nut_def.head(5)

,nutrient_id,units,tagname,description
0,203,g,PROCNT,Protein
1,204,g,FAT,Total lipid (fat)
2,205,g,CHOCDF,"Carbohydrate, by difference"
3,207,g,ASH,Ash
4,208,kcal,ENERC_KCAL,Energy


#### map rdi values with corresponding USDA nutrients

In [23]:
#try to map rdi elements with database elements automatically
mapping = {}
still_unmapped = list(rdi_nutrients)

for r in rdi_nutrients :
    
    mapped_count = 0
    sev_des = []
    
    for des in nut_def['description'].values :
        if ((r in des) or (des in r)) :
            
            sev_des.append(des)
            mapped_count += 1
            
            
    if mapped_count == 1 :
        mapping[r] = sev_des[0]
        still_unmapped.remove(r)
    
    elif mapped_count > 1:
        print("\nconflict for ", r , " : ")
        [print("\t- ", c) for c in sev_des]
        print("\n")
            
    else :
        print("No mapping for ", r)

print_bold("mapping found for the following nutrients : ")
mapping

No mapping for  alpha-linoleic acid
No mapping for  Biotin
No mapping for  Chloride
No mapping for  Chromium

conflict for  Fat  : 
	-  Fatty acids, total trans
	-  Fatty acids, total saturated
	-  Fatty acids, total monounsaturated
	-  Fatty acids, total polyunsaturated
	-  Fatty acids, total trans-monoenoic
	-  Fatty acids, total trans-polyenoic



conflict for  Folate  : 
	-  Folate, total
	-  Folate, food
	-  Folate, DFE


No mapping for  Iodine
No mapping for  Linoleic acid
No mapping for  Molybdenum
No mapping for  Monousaturated fat
No mapping for  Polyunsaturated fat
No mapping for  Saturated fat

conflict for  Vitamin A  : 
	-  Vitamin A, IU
	-  Vitamin A, RAE


No mapping for  Vitamin B6
No mapping for  Vitamin B12

conflict for  Vitamin D  : 
	-  Vitamin D
	-  Vitamin D2 (ergocalciferol)
	-  Vitamin D3 (cholecalciferol)
	-  Vitamin D (D2 + D3)



conflict for  Vitamin E  : 
	-  Vitamin E (alpha-tocopherol)
	-  Vitamin E, added


mapping found for the following nutrients : 


{'Calcium': 'Calcium, Ca',
 'Carbohydrate': 'Carbohydrate, by difference',
 'Choline': 'Choline, total',
 'Copper': 'Copper, Cu',
 'Fiber': 'Fiber, total dietary',
 'Fluoride': 'Fluoride, F',
 'Iron': 'Iron, Fe',
 'Magnesium': 'Magnesium, Mg',
 'Manganese': 'Manganese, Mn',
 'Niacin': 'Niacin',
 'Pantothenic acid': 'Pantothenic acid',
 'Phosphorus': 'Phosphorus, P',
 'Potassium': 'Potassium, K',
 'Protein': 'Protein',
 'Riboflavin': 'Riboflavin',
 'Selenium': 'Selenium, Se',
 'Sodium': 'Sodium, Na',
 'Sugar': 'Sugars, total',
 'Thiamin': 'Thiamin',
 'Vitamin C': 'Vitamin C, total ascorbic acid',
 'Vitamin K': 'Vitamin K (phylloquinone)',
 'Water': 'Water',
 'Zinc': 'Zinc, Zn'}

In [24]:
# solve conficlts manually
mapping['Folate'] = "Folate, total"
mapping['Vitamin A'] = "Vitamin A, RAE"
mapping['Vitamin D'] = "Vitamin D (D2 + D3)"
mapping["Vitamin E"] = "Vitamin E (alpha-tocopherol)"
mapping["Monounsaturated fat"] = "Fatty acids, total monounsaturated"
mapping["Polyunsaturated fat"] = "Fatty acids, total polyunsaturated"
mapping["Saturated fat"] = "Fatty acids, total saturated"
mapping['alpha-linoleic acid'] = "Alanine"
mapping["Vitamin B6"] = "Vitamin B-6"
mapping["Vitamin B12"] = "Vitamin B-12"
mapping['Fat'] = "Total lipid (fat)"

#we do not want to keep the USDA name, change it in the database
mapping['Linoleic acid'] = "Linoleic acid"
nut_def["description"] = nut_def["description"].replace("18:2 undifferentiated", "Linoleic acid")



conflicts_solved = ["Folate", "Vitamin A", "Vitamin D", "Vitamin E", "Saturated fat",
                   "Monounsaturated fat", "Polyunsaturated fat", "alpha-linoleic acid",
                   "Vitamin B6", "Vitamin B12", "Fat", "Linoleic acid"]

still_unmapped = [su for su in still_unmapped if (su not in conflicts_solved)]


print_bold("no mapping found for the following nutrients : ")

still_unmapped

no mapping found for the following nutrients : 


['Biotin',
 'Chloride',
 'Chromium',
 'Iodine',
 'Molybdenum',
 'Monousaturated fat']

#### change elements names in rdi data, add nutrient_id column

In [25]:
# change rdi elements names
rdi['nutrient'] = rdi['nutrient'].apply(lambda x : mapping[x] if x in mapping.keys() else x)
rdi = rdi[~(rdi['nutrient'].apply(lambda x : x in still_unmapped))]
rdi.set_index("nutrient", inplace=True)

#add nutrient_id column
#rdi["nutrient_id"] = rdi['element'].apply(lambda x : nut_def[nut_def['description'] == x]['nutrient_id'].values[0])

rdi.head()

#filter nut_def to keep only mapped elements
nut_def = nut_def[nut_def['description'].apply(lambda x : x in rdi.index.values)]

#### change RDI values to match USDA units

In [26]:
def change_value(unit, value) :
    
    if unit == 'mg' or unit == 'IU' :
        return value

    elif unit == 'g' :
        return (float(value) / 1000)
    
    else :
        return (float(value) * 1000)

    
new_male_rdis = []
new_female_rdis = []

for n in rdi.index.values :
    unit = nut_def[nut_def['description'] == n]['units'].values[0]
    male_rdi = rdi.loc[n]['Male_RDI(19-30)']
    female_rdi = rdi.loc[n]['Female_RDI(19-30)']
    new_male_rdis.append(change_value(unit, male_rdi))
    new_female_rdis.append(change_value(unit, female_rdi))
                  
    
#change values
rdi['Male_RDI(19-30)'] = pd.Series(new_male_rdis).values
rdi['Female_RDI(19-30)'] = pd.Series(new_female_rdis).values

#save new RDI file as csv
rdi.to_csv("./../generated/matching_rdi.csv")

rdi

,Male_RDI(19-30),Female_RDI(19-30)
nutrient,,
Alanine,1.6,1.1
"Calcium, Ca",1000.0,1000.0
"Carbohydrate, by difference",130.0,130.0
"Choline, total",550.0,425.0
"Copper, Cu",0.9,0.9
Total lipid (fat),65.0,65.0
"Fiber, total dietary",38.0,25.0
"Fluoride, F",4000.0,3000.0
"Folate, total",400.0,400.0


# load and filter nutrient data

#### load nutrient data

In [27]:
use_cols = [0, 1, 2]

columns = ["food_id", "nutrient_id", "nutr_per_100g"]

nut_data = pd.read_csv(nut_data_path, sep="^", encoding="ISO-8859-1", names=columns, usecols=use_cols, header=None)

nut_data.head()

,food_id,nutrient_id,nutr_per_100g
0,1001,208,717.0
1,1001,262,0.0
2,1001,263,0.0
3,1001,268,2999.0
4,1001,301,24.0


#### keep only nutrients that are included in RDI

In [28]:
print_bold("length before filtering : " + str(len(nut_data)))

#drop non-exploitable lines
nut_data = nut_data[nut_data['nutrient_id'].apply(lambda x : x in nut_def['nutrient_id'].values)]

print_bold("length after filtering : " + str(len(nut_data)))

#replace id by name to have more convenient reading
nut_data['nutrient'] = nut_data['nutrient_id'].apply(lambda x : nut_def[nut_def['nutrient_id'] == x]['description'].values[0])
nut_data = nut_data.drop("nutrient_id", axis=1)

length before filtering : 644125
length after filtering : 229821


#### keep only ids matching ingredients

In [29]:
m = json.load(open("./../generated/ing_id_mapping.json"))
matching_ids = [str(m[k]) for k in m]

print_bold("length before filtering : " + str(len(nut_data)))

#apply filtering
nut_data = nut_data[nut_data['food_id'].apply( lambda x : str(x) in matching_ids)]

print_bold("length after filtering : " + str(len(nut_data)))

length before filtering : 229821
length after filtering : 54984


In [30]:
#what average percentage of info do we have per mapped food ?
perc = 100 * nut_data.groupby('food_id').count().mean()[0] / len(rdi)
print_bold("percentage of available nutritional information : " + str(perc))

percentage of available nutritional information : 86.80496353130624


> Pretty nice ! 

#### add rdi percentage columns

In [31]:
nut_data["percentage_male_rdi"] = nut_data.apply(lambda row : 100*row['nutr_per_100g'] / rdi.loc[row['nutrient']]['Male_RDI(19-30)'], axis=1)
nut_data["percentage_female_rdi"] = nut_data.apply(lambda row : 100*row['nutr_per_100g'] / rdi.loc[row['nutrient']]['Female_RDI(19-30)'], axis=1)
nut_data.head()

,food_id,nutr_per_100g,nutrient,percentage_male_rdi,percentage_female_rdi
53,1003,4.0,"Calcium, Ca",0.400000,0.400000
54,1003,0.0,"Magnesium, Mg",0.000000,0.000000
55,1003,3.0,"Phosphorus, P",0.428571,0.428571
56,1003,5.0,"Potassium, K",0.106383,0.106383
57,1003,2.0,"Sodium, Na",0.133333,0.133333


#### pivot table to facilitate search by food id, save the resulting dataframe

In [32]:
#pivot table
nut_data = nut_data.pivot(index='food_id', columns='nutrient', values=['nutr_per_100g', 'percentage_male_rdi', 'percentage_female_rdi'])

#save table using h5 (easier for multi-index table storage)
nut_data.to_hdf('./../generated/nut_data.h5','table', append=True)

In [33]:
#check that the storage went fine
a = pd.read_hdf('./../generated/nut_data_3d.h5', 'table', where=['index>2'])
nut_data.loc[1003]['percentage_male_rdi']

nutrient
Alanine                                 0.625000
Calcium, Ca                             0.400000
Carbohydrate, by difference             0.000000
Choline, total                          4.054545
Copper, Cu                              0.111111
Fatty acids, total polyunsaturated     23.087500
Fatty acids, total saturated          516.033333
Fiber, total dietary                    0.000000
Fluoride, F                                  NaN
Folate, total                           0.000000
Iron, Fe                                0.000000
Linoleic acid                          13.217647
Magnesium, Mg                           0.000000
Manganese, Mn                           0.000000
Niacin                                  0.018750
Pantothenic acid                        0.200000
Phosphorus, P                           0.428571
Potassium, K                            0.106383
Protein                                 0.500000
Riboflavin                              0.384615
Selenium, S